## Preamble

In [3]:
# Setting up everything for you!

import os, sys
import pprint
from pathlib import Path

project_root = Path(os.getcwd()).parent
src_path     = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Hide the trackback for now

ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

# Import pyETM

from pyetm.config.settings import get_settings
from pyetm.models import Scenario

# Check if our API is ready!

print("Using ETM API at    ",       get_settings().base_url)
print("Token loaded?       ",       bool(get_settings().etm_api_token))

Using ETM API at     http://localhost:3000/api/v3
Token loaded?        True


## Picking my scenario

In [4]:
scenario = Scenario.load(2690506)

## Curves to pandas series

In [5]:
# Custom curves
if scenario.custom_curves.is_attached("interconnector_2_price"):
    print(
        scenario.custom_curve_series("interconnector_2_price")
    )

0       26.00
1       26.00
2       14.48
3        0.01
4        0.02
        ...  
8755     0.01
8756     0.01
8757     0.01
8758     0.02
8759     0.00
Name: interconnector_2_price, Length: 8760, dtype: float64


In [6]:
from pyetm.clients.base_client import BaseClient


curve = scenario.carrier_curves._find("electricity_price")
print(f"Curve found: {curve is not None}")

if curve:
    # Test direct curve retrieval
    direct_result = curve.retrieve(BaseClient(), scenario)
    print(f"Direct retrieve: {type(direct_result)}")

    # Test via get_contents
    contents_result = scenario.carrier_curves.get_contents(scenario, "electricity_price")
    print(f"get_contents: {type(contents_result)}")

    # Test via scenario method
    scenario_result = scenario.carrier_curve_series("electricity_price")
    print(f"scenario method: {type(scenario_result)}")


Curve found: True
Direct retrieve: <class 'pandas.core.frame.DataFrame'>
get_contents: <class 'pandas.core.frame.DataFrame'>
scenario method: <class 'pandas.core.frame.DataFrame'>


In [7]:
print(f"Available carrier curves: {list(scenario.carrier_curves.attached_keys())}")
result = scenario.carrier_curve_series("electricity_price")
print(f"Result: {type(result)}")
if result is not None:
    print(f"Shape: {result.shape}")
    print(f"First 5 values: {result.head()}")

Available carrier curves: ['merit_order', 'electricity_price', 'heat_network', 'agriculture_heat', 'household_heat', 'buildings_heat', 'hydrogen', 'network_gas', 'residual_load', 'hydrogen_integral_cost']
Result: <class 'pandas.core.frame.DataFrame'>
Shape: (8760, 1)
First 5 values:    Price (Euros)
0       5.450539
1       5.450539
2       5.450539
3       0.010000
4       0.010000


In [8]:
curve_count = 0
failed_curves = []

for curve_name in scenario.carrier_curves.attached_keys():
    print(f"Fetching {curve_name}...", end=" ")

    try:
        # This will fetch and cache the curves
        result = scenario.carrier_curve_series(curve_name)

        if result is not None:
            print(f"({result.shape[0]} points)")
            curve_count += 1
        else:
            print("No data")
            failed_curves.append(curve_name)

    except Exception as e:
        print(f"Error: {e}")
        failed_curves.append(curve_name)

print(f"   Successfully fetched: {curve_count} curves")
print(f"   Failed: {len(failed_curves)} curves")

if failed_curves:
    print(f"   Failed curves: {failed_curves}")


Fetching merit_order... (8760 points)
Fetching electricity_price... (8760 points)
Fetching heat_network... (8760 points)
Fetching agriculture_heat... (8760 points)
Fetching household_heat... (8760 points)
Fetching buildings_heat... (8760 points)
Fetching hydrogen... (8760 points)
Fetching network_gas... (8760 points)
Fetching residual_load... (8760 points)
Fetching hydrogen_integral_cost... (8760 points)
   Successfully fetched: 10 curves
   Failed: 0 curves


In [9]:
from pyetm.models import ScenarioPacker

packer = ScenarioPacker()

packer.add_inputs(scenario)
packer.add_custom_curves(scenario)
packer.add_carrier_curves(scenario)

print(packer.inputs())
print(packer.main_info())

# print(packer.custom_curves())


                                                   unit    value  default
climate_relevant_co2_biomass_gas_future               %      0.0      0.0
climate_relevant_co2_biomass_gas_present              %      0.0      0.0
climate_relevant_co2_biomass_liquid_future            %      0.0      0.0
climate_relevant_co2_biomass_liquid_present           %      0.0      0.0
climate_relevant_co2_biomass_solid_future             %      0.0      0.0
...                                                 ...      ...      ...
capacity_of_energy_battery_wind_turbine_inland       MW      0.0      0.0
capacity_of_energy_power_hybrid_wind_turbine_of...   MW     None      0.0
capacity_of_energy_power_wind_turbine_coastal        MW      0.0      0.0
capacity_of_energy_power_wind_turbine_inland         MW  10000.0  10000.0
capacity_of_energy_power_wind_turbine_offshore       MW  38000.0  38000.0

[1318 rows x 3 columns]
           2690506
end_year      2050
private      False
area_code   nl2019
template   

In [10]:
packer.to_excel('my_excel.xlsx')